# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
from dotenv import load_dotenv
load_dotenv(dotenv_path='../../01_materials/labs/.env')




True

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

import pandas as pd
dataset = os.getenv('PRICE_DATA')
files = glob(dataset)
data = [pd.read_parquet(f) for f in files]
merged_data = pd.concat(data,ignore_index=True)
# Write your code below.
merged_data


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,target
0,2000-01-03,16.625000,16.812500,16.125000,16.500000,5.365692,419600,Utilities,Electric Utilities,2000,9.900000,0.666667,1,1.0
1,2000-01-04,16.562500,17.125000,16.250000,17.062500,5.548609,623100,Utilities,Electric Utilities,2000,16.500000,0.034091,1,1.0
2,2000-01-05,17.000000,17.937500,17.000000,17.937500,5.833158,310400,Utilities,Electric Utilities,2000,17.062500,0.051282,1,0.0
3,2000-01-06,17.937500,17.937500,17.625000,17.812500,5.792508,283700,Utilities,Electric Utilities,2000,17.937500,-0.006969,0,0.0
4,2000-01-07,17.875000,17.937500,17.312500,17.625000,5.731534,225100,Utilities,Electric Utilities,2000,17.812500,-0.010526,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742198,2024-06-28,149.639999,149.639999,147.940002,149.039993,149.039993,3926000,Financials,Transaction & Payment Processing Services,2024,149.080002,-0.000268,0,0.0
2742199,2024-07-01,149.550003,150.070007,147.050003,147.139999,147.139999,1572800,Financials,Transaction & Payment Processing Services,2024,149.039993,-0.012748,0,1.0
2742200,2024-07-02,146.770004,149.460007,146.770004,149.389999,149.389999,1480400,Financials,Transaction & Payment Processing Services,2024,147.139999,0.015292,1,0.0
2742201,2024-07-03,149.419998,149.970001,148.160004,148.789993,148.789993,1142600,Financials,Transaction & Payment Processing Services,2024,149.389999,-0.004016,0,1.0


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import dask.dataframe as dd
# Add lag columns for Close and Adj_Close
merged_data['Close_lag'] = merged_data['Close'].shift(1)
merged_data['Adj_Close_lag'] = merged_data['Adj Close'].shift(1)
merged_data['returns'] = (merged_data['Adj Close'] / merged_data['Adj_Close_lag']) - 1
merged_data['hi_lo_range'] = merged_data['High'] - merged_data['Low']
dd_feat = dd.from_pandas(merged_data)
result=dd_feat.compute()



+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
del merged_data
merged_data=pd.DataFrame(result)
merged_data['rolling_avg_return'] = merged_data['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?
* It was not necessary to convert to pandas here. 
* Directly using dask would have been better because we're dealing with data that's quite large ( 25k+ rows ) dask allows us to parallelize the computation and take advantage of better hardware,
(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.